In [3]:
from selenium import webdriver
import pandas as pd
import random
import time
from lxml import etree
import requests
from urllib.parse import quote
import time

In [4]:
from bs4 import BeautifulSoup as bs

In [5]:
url = 'https://www.reddit.com/r/Bitcoin/new/'

In [8]:
import os
import sys
os.path.dirname(sys.executable)

'/usr/bin'

In [9]:
options = webdriver.ChromeOptions()

In [13]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

Executing: /tmp/apt-key-gpghome.6O9azn6yyW/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.ecBKFnpRbS/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.spo3rIooGG/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [80]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
wd = webdriver.Chrome('chromedriver',options=chrome_options)
wd.get('https://www.reddit.com/r/Bitcoin/new/')

<ipython-input-80-20798f60973e>:4: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


In [81]:
for i in range(100):
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

In [82]:
content = wd.page_source
webpage_soup1 = bs(content)

In [84]:
title = []
text = []
time_ = []
user = []
vote = []
comment_number = []
for i in webpage_soup1.find_all(name = 'div', attrs={'data-click-id':'background'}):
    title.append(i.find(name = 'h3', attrs='_eYtD2XCVieq6emjKBH3m').string)
    try:
        text.append(i.find(name = 'div', attrs='_292iotee39Lmt0MkQZ2hPV RichTextJSON-root').text)
    except:
        text.append('')
    try:
        time_.append(i.find(name = 'span', attrs={'data-click-id':'timestamp'}).text)
    except:
        time_.append('')
    try:
        user.append(i.find(name = 'a', attrs={'data-click-id':'user'}).text)
    except:
        user.append('')
    vote.append(i.find(name = 'div', attrs='_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ').string)
    comment_number.append(i.find(name = 'span', attrs='FHCV02u6Cp2zYL0fhQPsO').string)

In [85]:
df = pd.DataFrame()
df['title'] = title
df['text'] = text
df['time_'] = time_
df['user'] = user
df['vote'] = vote
df['comment_number'] = comment_number

In [87]:
df

,title,text,time_,user,vote,comment_number
0,The absolutely perfect use case for running Bi...,,7 minutes ago,u/CyberdyneLabs,Vote,0 comments
1,Number goes up - Lightning Network State,,17 minutes ago,u/Up1now,Vote,0 comments
2,Dubai economic free zone to invest an extra $5...,,36 minutes ago,u/CryptoKingSA,Vote,1 comment
3,How To Increase The Price Of Bitcoin,,54 minutes ago,u/Jon_Hodl,Vote,2 comments
4,RİP.... You will be remembered,,10 hours ago,u/kazo66666,498,54 comments
...,...,...,...,...,...,...
95,What are your percentages on Bitcoin DCA?,I am DCAing since January 2022 weekly every Tu...,2 days ago,u/Mallukotti,2,24 comments
96,BTC Graphic Designer Offering his Services to ...,"Hello Reddit!My name is Raul Bedoya Ching, and...",2 days ago,u/megabedoya,9,6 comments
97,empty block mined,https://blockstream.info/block/000000000000000...,2 days ago,u/antennawire,0,20 comments
98,Bitcoin Miners Are Playing a High-Stakes Game ...,,2 days ago,u/Bitcoinbobbie,0,1 comment


In [75]:
from google.colab import files
df.to_csv('output.csv', index=False) 
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>